# 0. Warm up
Get started with PyTorch and play with numbers

In [ ]:
import torch

In [ ]:
a = torch.randn(3)

In [ ]:
print(a.shape)
print(a)

In [ ]:
a3 = a * 3.0

In [ ]:
print(a3)

In [ ]:
for i in range(3):
    print(i, a[i], a[i]*3.0, a3[i])

In torch, the matrix multiplication is the `mm` function. See [here](https://stackoverflow.com/questions/44524901/how-to-do-product-of-matrices-in-pytorch)

To compute $A \times B$, where $A$ and $B$ are matrices, you should use 
```[python]
A.mm(B) # or
torch.mm(A, B)
```

In [ ]:
m = torch.randn(2, 3)

In [ ]:
torch.mm(m, a.reshape(3,1))

In [ ]:
s = 0
for i in range(3):
    s += m[0][i] * a[i]
print(s)

# 1. Get some data

In [ ]:
import torchvision
from torchvision import transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(
    root='../data', train=True, download=True,
    transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32,
    shuffle=True, num_workers=2
)

In [ ]:
for X, y in trainloader:
    break

In [ ]:
print(X.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def show(img):
    npimg = img.detach().numpy()
    npimg -= npimg.min()
    npimg /= npimg.max()
    if npimg.shape[0] in [3,4]:
        plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    else:
        plt.imshow(npimg.squeeze(), interpolation='nearest', cmap='gray')
        

In [ ]:
show(X[1])

In [ ]:
print(X[0, 0, 10:20, 10:20])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3)

conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)


In [ ]:
class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=64, 
                               kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, 
                               out_channels=128, 
                               kernel_size=3)
        self.linear = nn.Linear(3200, 10)
        
    def forward(self, x):
        """
        :param x: a batch of images
        """
        h = self.conv1(x)
        h = F.leaky_relu(h, 0.2, inplace=True)
        h = F.max_pool2d(h, 2)
        h = self.conv2(h)
        h = F.leaky_relu(h, 0.2, inplace=True)
        h = F.max_pool2d(h, 2)
        h = self.linear(h.view(h.shape[0], 3200))
        h = F.log_softmax(h, dim=1)
        return h
        
    
    

In [ ]:
dnet = DNet()

In [ ]:
h = dnet(X)

In [ ]:
print(h.shape)

In [ ]:
print(h[0])

In [ ]:
F.nll_loss(h, y)

In [ ]:
h1 = h
h1[0][2]+=0.1
print(F.nll_loss(h1,y))

In [ ]:
from torch.optim import Adam

In [ ]:
optim = Adam(dnet.parameters(), lr=1e-4)

In [ ]:
optim.zero_grad()
h = dnet(X)
loss = F.nll_loss(h, y)
loss.backward()
optim.step()
print(loss)

### Put together

In [ ]:
# To perform training on the entire data
optim = Adam(dnet.parameters(), lr=1e-4)
epoch = 0
while epoch < 100:
    it = 0
    for X, y in trainloader:
        optim.zero_grad()
        h = dnet(X)
        loss = F.nll_loss(h, y)
        loss.backward()
        optim.step()
        it += 1
        if it%100==0:
            print("Epoch {}, iteration {} train loss {:.3f}".format(
                epoch, it, loss))
        
            
    # you can validate the model on test data here, try